In [4]:
import pandas as pd
import glob
import os
from tqdm import tqdm_notebook as tqdm
#preprocessing
dirlist = ["dokujo-tsushin","it-life-hack","kaden-channel","livedoor-homme",
           "movie-enter","peachy","smax","sports-watch","topic-news"]
df = pd.DataFrame(columns=["label","class","news"])
for index , i in enumerate(tqdm(dirlist)):
    path = "/home/jovyan/work/livedoor/text/"+i+"/*.txt" 
    files = glob.glob(path)
    files.pop()
    for j in tqdm(files):
        f = open(j,encoding="utf-8")
        data = f.read() 
        f.close()
        t = pd.Series(['__label__'+i,i,"".join(data.split("\n")[3:])],index = df.columns)
        df  = df.append(t,ignore_index=True)

In [23]:
t

label                                  __label__topic-news
class                                           topic-news
news     19日にダルビッシュ有とモデル・紗栄子の離婚成立が報じられたが、同日「5時に夢中！」（東京M...
dtype: object

In [7]:
import logging
import numpy as np
from gensim.models import Word2Vec
import MeCab
import time
from sklearn.preprocessing import normalize
import sys
import re

start = time.time()
tokenizer =  MeCab.Tagger("-Owakati")  
sentences = []
hisaka = []
print ("Parsing sentences from training set...")

# Loop over each news article.
for review in tqdm(df["news"]):
    try:
        # Split a review into parsed sentences.
        result = tokenizer.parse(review).replace("\u3000","").replace("\n","")
        result = re.sub(r'[0123456789０１２３４５６７８９！＠＃＄％＾＆\-|\\＊\“（）＿■×※⇒—●(：〜＋=)／*&^%$#@!~`){}…\[\]\"\'\”:;<>?＜＞？、。・,./『』【】「」→←○]+', "", result)
        h = result.split(" ")
        h = list(filter(("").__ne__, h))
        sentences.append(result)
    except:
        continue

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

df['news'] = sentences
df.to_csv("/home/jovyan/work/livedoor/text/livedoor_fastText2.csv",columns=['label', 'news'],index=False,header=False )

Parsing sentences from training set...


In [29]:
df['label'][0]

'__label__dokujo-tsushin'

In [53]:
from gensim.test.utils import common_texts
from gensim.models import FastText
from gensim.models.word2vec import LineSentence
import pandas as pd

sample = LineSentence("/home/jovyan/work/livedoor/text/livedoor_fastText2.csv")
model = FastText(sample, size=4, window=3, min_count=1, iter=10)



2018-12-17 05:38:44,335 : INFO : collecting all words and their counts
2018-12-17 05:38:44,337 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-12-17 05:38:45,457 : INFO : collected 74372 word types from a corpus of 3917070 raw words and 7367 sentences
2018-12-17 05:38:45,457 : INFO : Loading a fresh vocabulary
2018-12-17 05:38:45,745 : INFO : effective_min_count=1 retains 74372 unique words (100% of original 74372, drops 0)
2018-12-17 05:38:45,746 : INFO : effective_min_count=1 leaves 3917070 word corpus (100% of original 3917070, drops 0)
2018-12-17 05:38:46,099 : INFO : deleting the raw counts dictionary of 74372 items
2018-12-17 05:38:46,100 : INFO : sample=0.001 downsamples 33 most-common words
2018-12-17 05:38:46,101 : INFO : downsampling leaves estimated 2902050 word corpus (74.1% of prior 3917070)
2018-12-17 05:38:47,189 : INFO : estimated required memory for 74372 words, 446339 buckets and 4 dimensions: 58928448 bytes
2018-12-17 05:38:47,207 : IN

In [54]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("fasttext.model")
model.save(fname)
model = FastText.load(fname)  

2018-12-17 05:39:44,275 : INFO : saving FastText object under /tmp/fasttext.model, separately None
2018-12-17 05:39:44,276 : INFO : not storing attribute vectors_norm
2018-12-17 05:39:44,278 : INFO : not storing attribute vectors_vocab_norm
2018-12-17 05:39:44,279 : INFO : not storing attribute vectors_ngrams_norm
2018-12-17 05:39:44,280 : INFO : not storing attribute buckets_word
2018-12-17 05:39:44,653 : INFO : saved /tmp/fasttext.model
2018-12-17 05:39:44,655 : INFO : loading FastText object from /tmp/fasttext.model
2018-12-17 05:39:45,013 : INFO : loading wv recursively from /tmp/fasttext.model.wv.* with mmap=None
2018-12-17 05:39:45,015 : INFO : setting ignored attribute vectors_norm to None
2018-12-17 05:39:45,015 : INFO : setting ignored attribute vectors_vocab_norm to None
2018-12-17 05:39:45,016 : INFO : setting ignored attribute vectors_ngrams_norm to None
2018-12-17 05:39:45,016 : INFO : setting ignored attribute buckets_word to None
2018-12-17 05:39:45,017 : INFO : loading 

In [61]:
existent_word = "事件"
existent_word in model.wv.vocab


True

In [74]:
not_matching = model.wv.doesnt_match("human computer interface tree".split())
sim_score = model.wv.similarity('computer', 'human')

In [75]:
not_matching

'interface'

In [76]:
sim_score

0.98126632

In [77]:
similarities = model.wv.most_similar(positive=['computer', 'human'], negative=['interface'])
most_similar = similarities[0]
most_similar

('師範', 0.9993112087249756)